In [1]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_culture_events = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?s ?event ?o ?site ?urlcity ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [2]:
df = sparql_dataframe.get(endpoint, query_culture_events)

In [12]:
print(df)

                                                       s  \
0      http://dati.beniculturali.it/mibact/eventi/res...   
1      http://dati.beniculturali.it/mibact/eventi/res...   
2      http://dati.beniculturali.it/mibact/eventi/res...   
3      http://dati.beniculturali.it/mibact/eventi/res...   
4      http://dati.beniculturali.it/mibact/eventi/res...   
...                                                  ...   
21163  http://dati.beniculturali.it/mibact/eventi/res...   
21164  http://dati.beniculturali.it/mibact/eventi/res...   
21165  http://dati.beniculturali.it/mibact/eventi/res...   
21166  http://dati.beniculturali.it/mibact/eventi/res...   
21167  http://dati.beniculturali.it/mibact/eventi/res...   

                                                   event  \
0      “Un brindisi per l’eternità”, all'Antiquarium ...   
1      Giornate Europee del Patrimonio 2022 al Museo ...   
2              La storia ragionevole dei Bronzi di Riace   
3      Osservando le mura aureliane: un

In [4]:
df=df.drop_duplicates(["s", "o"])

In [3]:
df.to_csv("cleancefile.csv")

OSError: Cannot save file into a non-existent directory: '/Users/martinapensalfini/Desktop/infowiz'

In [25]:
df_city=df["city"]
print(df_city)

0                    Numana
1                     Lecce
2        Reggio di Calabria
3                      Roma
4        Reggio di Calabria
                ...        
21163               Venezia
21164                 Sarno
21165               Bologna
21166               Bologna
21167               Bologna
Name: city, Length: 21101, dtype: object


In [26]:
counts = dict()
for i in df["city"]:
    if i=="Bologna":
        counts[i] = counts.get(i, 0) + 1
print(counts)


{'Bologna': 198}


In [27]:
counts = dict()
for i in df["city"]:
    if i=="Zogno":
        counts[i] = counts.get(i, 0) + 1
print(counts)

{'Zogno': 2}


In [28]:
counts = dict()
for i in df["city"]:
    if i=="Amalfi":
        counts[i] = counts.get(i, 0) + 1
print(counts)

{'Amalfi': 2}


In [29]:
counts = dict()
for i in df["city"]:
    if i=="Buddusò":
        counts[i] = counts.get(i, 0) + 1
print(counts)

{'Buddusò': 1}


In [6]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_site = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT (count (?o) as ?count) ?site WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [7]:
df = sparql_dataframe.get(endpoint, query_site)

In [11]:
df.to_csv("numbersite.csv")

In [10]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_city = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT (count (?s) as ?count) ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [12]:
df = sparql_dataframe.get(endpoint, query_city)

In [13]:
df.to_csv("numbercity.csv")

In [15]:
endpoint = 'https://dati.cultura.gov.it/sparql'

query = '''
SELECT DISTINCT(?urlcity2) ?dbCity WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbCity. 
FILTER(contains (str(?dbCity), "dbpedia") )
}
'''

In [16]:
df = sparql_dataframe.get(endpoint, query)

In [17]:
df

,urlcity2,dbCity
0,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Besenello
1,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Calci
2,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Carmignano
3,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Cetona
4,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Chiusi
...,...,...
789,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Casalciprano
790,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Monterosso_Grana
791,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Castrocielo
792,http://dati.beniculturali.it/mibact/luoghi/res...,http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...


In [2]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_time="""SELECT DISTINCT ?s ?time ?starttime ?endtime WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s tiapit:atTime ?time.
?time tiapit:startTime ?starttime;
tiapit:endTime ?endtime.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}"""

In [6]:
df = sparql_dataframe.get(endpoint, query_time)

In [7]:
df

,s,time,starttime,endtime
0,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-09-29 00:00:00,2022-09-29 00:00:00
1,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-09 00:00:00,2022-10-09 00:00:00
2,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-09 00:00:00,2022-12-31 00:00:00
3,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-08 00:00:00,2022-10-09 00:00:00
4,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-10-09 00:00:00,2022-10-09 00:00:00
...,...,...,...,...
21109,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-12-03 00:00:00,2022-12-03 00:00:00
21110,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-11-30 00:00:00,2022-11-30 00:00:00
21111,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-11-25 00:00:00,2022-11-25 00:00:00
21112,http://dati.beniculturali.it/mibact/eventi/res...,http://dati.beniculturali.it/mibact/eventi/res...,2022-11-27 00:00:00,2022-11-27 00:00:00


In [8]:
df.to_csv("time.csv")

In [9]:
query_count_time ="""SELECT DISTINCT (count (?s) as ?count) ?starttime WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s tiapit:atTime ?time.
?time tiapit:startTime ?starttime;
tiapit:endTime ?endtime.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}"""

In [10]:
df = sparql_dataframe.get(endpoint, query_count_time)

In [11]:
df

,count,starttime
0,3,2022-12-30 00:00:00
1,1,2022-09-19 00:00:00
2,2,2023-02-19 00:00:00
3,3,2023-01-24 00:00:00
4,4,2023-01-30 00:00:00
...,...,...
1258,28,2022-01-27 00:00:00
1259,6,2022-02-05 00:00:00
1260,4,2020-02-03 00:00:00
1261,11,2020-02-21 00:00:00


In [12]:
df.to_csv("time_count.csv")

In [3]:
query_institutions ="""SELECT DISTINCT ?topic WHERE {
?s foaf:primaryTopic ?topic
}"""

df = sparql_dataframe.get(endpoint, query_institutions)

In [4]:
df

,topic
0,http://dati.beniculturali.it/mibact/luoghi/res...
1,http://dati.beniculturali.it/mibact/luoghi/res...
2,http://dati.beniculturali.it/mibact/luoghi/res...
3,http://dati.beniculturali.it/mibact/luoghi/res...
4,http://dati.beniculturali.it/mibact/luoghi/res...
...,...
27560,http://dati.beniculturali.it/mibact/eventi/res...
27561,http://dati.beniculturali.it/mibact/eventi/res...
27562,http://dati.beniculturali.it/mibact/eventi/res...
27563,http://dati.beniculturali.it/mibact/eventi/res...


In [5]:
df.to_csv("institutions.csv")

In [6]:
query_lat_long ="""SELECT DISTINCT ?topic ?lat ?long WHERE {
?s foaf:primaryTopic ?topic.
?topic geo:lat ?lat.
?topic geo:long ?long.
}"""

df = sparql_dataframe.get(endpoint, query_lat_long)

In [7]:
df

,topic,lat,long
0,http://dati.beniculturali.it/mibact/luoghi/res...,46.500280,11.343991
1,http://dati.beniculturali.it/mibact/luoghi/res...,46.073410,11.112929
2,http://dati.beniculturali.it/mibact/luoghi/res...,41.891030,12.487999
3,http://dati.beniculturali.it/mibact/luoghi/res...,42.416588,13.079355
4,http://dati.beniculturali.it/mibact/luoghi/res...,41.561714,14.657649
...,...,...,...
5912,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359
5913,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359
5914,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359
5915,http://dati.beniculturali.it/mibact/luoghi/res...,41.869560,12.568359


In [8]:
df.to_csv("latlonginstitutions.csv")